##Corpus Streaming – One Document at a Time

Note that corpus above resides fully in memory, as a plain Python list. In this simple example, it doesn’t matter much, but just to make things clear, let’s assume there are millions of documents in the corpus. Storing all of them in RAM won’t do. Instead, let’s assume the documents are stored in a file on disk, one document per line. Gensim only requires that a corpus be able to return one document vector at a time:

In [7]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
import os
import tempfile
TEMP_FOLDER = tempfile.gettempdir()
print('Folder "{}" will be used to save temporary dictionary and corpus.'.format(TEMP_FOLDER))
from gensim import corpora

# collect statistics about all tokens
dictionary = corpora.Dictionary(line.lower().split() for line in open('datasets/mycorpus.txt'))

class MyCorpus(object):
    def __iter__(self):
        for line in open('datasets/mycorpus.txt'):
            # assume there's one document per line, tokens separated by whitespace
            yield dictionary.doc2bow(line.lower().split())
            
corpus_memory_friendly = MyCorpus() # doesn't load the corpus into memory!
print(corpus_memory_friendly)

for vector in corpus_memory_friendly:  # load one vector into memory at a time
    print(vector)
    


2017-10-06 17:56:01,589 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2017-10-06 17:56:01,591 : INFO : built Dictionary(42 unique tokens: ['perceived', 'interface', 'abc', 'of', 'for']...) from 9 documents (total 69 corpus positions)


Folder "/tmp" will be used to save temporary dictionary and corpus.
[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)]
[(3, 1), (8, 2), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1)]
[(6, 1), (13, 1), (14, 1), (16, 1), (17, 1), (18, 1)]
[(1, 1), (8, 1), (13, 2), (17, 1), (19, 1), (20, 1), (21, 1)]
[(8, 1), (12, 1), (14, 1), (15, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1)]
[(8, 1), (18, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1)]
[(8, 1), (18, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1)]
[(8, 1), (21, 1), (31, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1)]
[(9, 1), (11, 1), (35, 1), (41, 1)]


We are going to create the dictionary from the mycorpus.txt file without loading the entire file into memory. Then, we will generate the list of token ids to remove from this dictionary by querying the dictionary for the token ids of the stop words, and by querying the document frequencies dictionary (dictionary.dfs) for token ids that only appear once. Finally, we will filter these token ids out of our dictionary. Keep in mind that dictionary.filter_tokens (and some other functions such as dictionary.add_document) will call dictionary.compactify() to remove the gaps in the token id series thus enumeration of remaining tokens can be changed.

In [8]:
from six import iteritems

# remove common words and tokenize
stoplist = set('for a of the and to in'.split())

# remove stop words and words that appear only once
stop_ids = [dictionary.token2id[stopword] for stopword in stoplist 
            if stopword in dictionary.token2id]
once_ids = [tokenid for tokenid, docfreq in iteritems(dictionary.dfs) if docfreq == 1]

# remove stop words and words that appear only once
dictionary.filter_tokens(stop_ids + once_ids)
print(dictionary)

Dictionary(12 unique tokens: ['interface', 'eps', 'computer', 'time', 'user']...)
